### **Import Libraries & Setup**

In [156]:
# Import Library
import pandas as pd
import numpy as np
import os 
from src.utils import extract_multi_csv
from config.setting import PRODUCTS_PATH

### **Data Ingestion**

In [157]:
# Ingest Data
file_list = os.listdir(PRODUCTS_PATH)

# Extract Data Product 
df_product_original = extract_multi_csv(PRODUCTS_PATH)

# Membuat Staging Area
df_products = df_product_original.copy()
df_products.drop(columns='Unnamed: 0', inplace=True)

# Preview Data
df_products.head()


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990"
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990"
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990"
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990"
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790"


### **Data Profiling**

In [158]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   name            1103170 non-null  object
 1   main_category   1103170 non-null  object
 2   sub_category    1103170 non-null  object
 3   image           1103170 non-null  object
 4   link            1103170 non-null  object
 5   ratings         751582 non-null   object
 6   no_of_ratings   751582 non-null   object
 7   discount_price  980844 non-null   object
 8   actual_price    1067544 non-null  object
dtypes: object(9)
memory usage: 75.7+ MB


In [159]:
df_products.isnull().sum()

name                   0
main_category          0
sub_category           0
image                  0
link                   0
ratings           351588
no_of_ratings     351588
discount_price    122326
actual_price       35626
dtype: int64

### **Data Cleaning & Handling Missing**

In [160]:
df_products['ratings'].unique()

array(['4.2', '4.0', '4.1', '4.3', '3.9', '3.8', '3.5', nan, '4.6', '3.3',
       '3.4', '3.7', '2.9', '5.0', '4.4', '3.6', '2.7', '4.5', '3.0',
       '3.1', '3.2', '4.8', '4.7', '2.5', '1.0', '2.6', '2.8', '2.3',
       '1.7', 'Get', '1.8', '2.4', '4.9', '2.2', '1.6', '1.9', '2.0',
       '1.4', '2.1', 'FREE', '1.2', '1.3', '1.5', '₹68.99', '₹65', '1.1',
       '₹70', '₹100', '₹99', '₹2.99', 4.0, 3.7, 3.4, 4.1, 3.8, 3.6, 4.2,
       3.9, 3.2, 3.5, 5.0, 3.3, 2.6, 4.5, 4.3, 4.7, 3.0, 3.1, 4.4, 4.6,
       1.0, 4.9, 2.9, 2.7, 2.2, 2.5, 4.8, 2.0, 1.7, 1.9, 2.8, 2.4, 1.6,
       2.3, 1.8, 1.4, 1.3, 2.1, 1.5], dtype=object)

Kolom **`ratings`** masih berisi data yang tidak seragam, seperti teks non-numerik ("FREE", "Get") dan nilai berbentuk mata uang ("₹68.99", "₹100"), serta campuran tipe string dan float. Agar dapat dianalisis, kolom ini perlu dibersihkan dengan menghapus nilai tidak valid, membatasi hanya angka dalam rentang 1.0–5.0, lalu mengubahnya ke format numerik `float`.

In [161]:
# mengubah format jadi dari object jadi numeric
df_products['ratings'] = pd.to_numeric(df_products['ratings'], errors='coerce').astype(float)

In [162]:
# Preview Hasil
df_products['ratings'].unique()

array([4.2, 4. , 4.1, 4.3, 3.9, 3.8, 3.5, nan, 4.6, 3.3, 3.4, 3.7, 2.9,
       5. , 4.4, 3.6, 2.7, 4.5, 3. , 3.1, 3.2, 4.8, 4.7, 2.5, 1. , 2.6,
       2.8, 2.3, 1.7, 1.8, 2.4, 4.9, 2.2, 1.6, 1.9, 2. , 1.4, 2.1, 1.2,
       1.3, 1.5, 1.1])

In [163]:
df_products['no_of_ratings'].unique() 

array(['2,255', '2,948', '1,206', ..., 5.0, 15.0, 8.0],
      shape=(8381,), dtype=object)

sama halnya dengan kolom 'no_of_ratings' perlu di format dari 'object' menjadi 'numeric'

In [164]:
# mengubah jadi numeric
df_products['no_of_ratings'] = pd.to_numeric(df_products['no_of_ratings'].str.replace(",", ""),errors='coerce')
df_products['no_of_ratings'] = df_products['no_of_ratings'].fillna(0)
df_products['no_of_ratings'] = df_products['no_of_ratings'].astype('Int64')

In [165]:
df_products['no_of_ratings'].unique() 

<IntegerArray>
[ 2255,  2948,  1206,    69,   630,  1666,  1097,  1494,   674,   801,
 ...
  6216,  5916,  3116,  5183, 67255,  5603, 55051,  3329,  7141,  4406]
Length: 8285, dtype: Int64

In [166]:
df_products['actual_price'].unique()

array(['₹58,990', '₹75,990', '₹61,990', ..., '₹608.97', '₹4,792',
       '₹8,023.60'], shape=(23171,), dtype=object)

In [167]:
df_products['discount_price'].unique()

array(['₹32,999', '₹46,490', '₹34,490', ..., '₹3,712.10', '₹1,429.60',
       '₹651.01'], shape=(27512,), dtype=object)

Dataset memiliki dua kolom utama terkait harga:  

- **`discount_price`** → harga produk setelah diskon, masih dalam format string dengan simbol mata uang (`₹`) dan tanda pemisah ribuan (`,`).  
- **`actual_price`** → harga asli produk sebelum diskon, juga masih berupa string dengan simbol mata uang dan pemisah ribuan.  

Agar dapat dianalisis lebih lanjut, diperlukan beberapa langkah pembersihan data:  

1. Membuat kolom baru bernama **`currency`** untuk menyimpan simbol mata uang (misalnya `₹`, `$`, `Rp`).  
2. Menghapus simbol mata uang dan tanda pemisah ribuan dari kolom harga.  
3. Mengonversi nilai harga ke tipe numerik (`float64`).  

Dengan cara ini, informasi harga tetap terjaga dan analisis lintas mata uang dapat dilakukan dengan lebih fleksibel.  

Kolom discount_price masih berisi nilai harga dalam format string yang disertai simbol mata uang (₹) serta tanda pemisah ribuan (,). Kondisi ini membuat kolom belum bisa diolah sebagai data numerik. Agar dapat dianalisis, langkah yang perlu dilakukan adalah membersihkan data dengan cara menghapus simbol mata uang dan tanda pemisah, lalu mengonversinya ke tipe numerik (float64). Dengan begitu, nilai harga dapat digunakan untuk perhitungan statistik maupun analisis perbandingan harga.

In [168]:
# Discount_Price
df_products['discount_price'] = pd.to_numeric(df_products['discount_price'].str.replace(r'[₹,]','', regex=True),errors='coerce')

# Actual Price
df_products['currency']= df_products['actual_price'].str[:1] # membuat kolom baru currency "₹"
df_products['actual_price'] = df_products['actual_price'].str.replace(r'[₹,]','', regex=True).astype(float)

Setelah dianalisis, ternyata memang beberapa row memiliki harga 0, kita akan mengubahnya untuk menjadi NaN.

In [169]:
df_products['actual_price'] = df_products['actual_price'].replace(0,np.nan)

In [184]:
# Membuat discount_percentage
df_products['discount_percentage'] = ((df_products['actual_price'] - df_products['discount_price']) / df_products['actual_price'] * 100).round(1)

In [187]:
# Preview 'df_product'
df_products.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,currency,discount_percentage
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,2255,32999.0,58990.0,₹,44.1
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,2948,46490.0,75990.0,₹,38.8
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,1206,34490.0,61990.0,₹,44.4
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,37990.0,68990.0,₹,44.9
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,34490.0,67790.0,₹,49.1
